In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import multiprocessing as mp
import torchvision
import timm

from matplotlib import pyplot as plt
from PIL import Image
from tempfile import TemporaryDirectory
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torchvision.transforms.functional import InterpolationMode
from transformers import get_cosine_schedule_with_warmup


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="F2aSznx6vyycTtjQ4IqS")
project = rf.workspace("ayurved").project("ayurved")
dataset = project.version(2).download("folder")

In [ ]:
# OS ENV SETUP

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# CONFIG


DATA_DIR = '/kaggle/working/Ayurved-2'
TEST_DIR = '/kaggle/working/Ayurved-2/test/Test'
TRAIN_DIR ='/kaggle/working/Ayurved-2/train'
SUB_DIR = '/kaggle/working/image_species.csv'
W_PATH = '/kaggle/input/test-file-model/ayurmodelstate.pth'
SEED = 279
TRAIN_BS = 8
TEST_BS = 100
NUM_CLASSES = 6
EMBEDDING_SIZE = 1280
NUM_EPOCHS = 14
LEARNING_RATE = 0.0003
WEIGHT_DECAY = 0
WARMUP_EPOCHS = 0
LOGGING_INTERVAL = 100
N_CORES = mp.cpu_count()


In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device(type='cuda')
else:
    DEVICE = torch.device('cpu')

print(f'USING device: {DEVICE}')


In [ ]:
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


In [ ]:
set_seed()

In [ ]:
class_labels = os.listdir(TRAIN_DIR)
print(class_labels)

In [ ]:
test_files = os.listdir(TEST_DIR)
test_df = pd.DataFrame({'file_path': test_files})
test_df['label'] = 9999

In [ ]:
# COnfiguring Data Set

class PlantDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.file_path = df['file_path']
        self.y = df['label']

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir, self.file_path[index]))
        if self.transform is not None:
            img = self.transform(img)
        label = self.y[index]
        return img, label

    def __len__(self):
        return self.y.shape[0]


In [ ]:
def convert_4_channel_to_3_channel(image):
    """
    Convert 4-channel RGBA image to 3-channel RGB image
    """
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    return image


data_transforms = {
    'train':transforms.Compose([
    transforms.Lambda(convert_4_channel_to_3_channel),
    transforms.Resize(
        size=(250, 250), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomRotation(degrees=(-180, 180)),
    transforms.RandomAffine(degrees=0, translate=(0.3, 0.3), scale=(0.7, 1.3)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
]),
    'valid':transforms.Compose([
    transforms.Lambda(convert_4_channel_to_3_channel),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
    }
test_transforms = transforms.Compose([
    transforms.Lambda(convert_4_channel_to_3_channel), 
    transforms.Resize(size=(224, 224), interpolation=InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
])

In [ ]:
test_dataset = PlantDataset(df=test_df,
                           img_dir=TEST_DIR,
                           transform=test_transforms)
test_loader = DataLoader(dataset=test_dataset,
                        batch_size=TEST_BS,
                        drop_last=False,
                        shuffle=False,
                        num_workers=N_CORES)

In [ ]:
img_dataset = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in ['train', 'valid']}

In [ ]:
dataloader ={x:DataLoader(img_dataset[x], batch_size=TRAIN_BS, drop_last=True, shuffle=True, num_workers=N_CORES) for x in ['train','valid']}

In [ ]:
dataset_sizes = {x: len(img_dataset[x]) for x in ['train', 'valid']}
class_names = img_dataset['train'].classes

In [ ]:
class EffNet(nn.Module):
    def __init__(self, num_classes, embedding_size):
        super(EffNet, self).__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.backbone = timm.create_model(
            'efficientnet_b1',
            pretrained=True,
            num_classes=self.num_classes
        )
        
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.embedding_size,256),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Linear(128, self.num_classes)
            )
        
    def forward(self,x):
        return self.backbone(x)


In [ ]:
set_seed(SEED)
model = EffNet(num_classes=NUM_CLASSES, embedding_size=EMBEDDING_SIZE)
model.to(DEVICE)

# with open('model_state.pt', 'wb') as f: 
#         torch.save(model().state_dict(), f)

optimizer = optim.AdamW(
    model.parameters(), 
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=WARMUP_EPOCHS,
    num_training_steps=dataset_sizes['train']*NUM_EPOCHS
)

scaler = GradScaler()

In [ ]:
def train_model(model,  optimizer, scaler, scheduler, num_epochs):
    
    since = time.time()

    
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'model_state.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            
            for phase in ['train', 'valid']:
                if phase == 'train':
                    model.train()  
                else:
                    model.eval()   

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloader[phase]:
                    inputs = inputs.to(DEVICE)
                    labels = labels.to(DEVICE)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        with autocast():
                            logits = model(inputs)
                            _, preds = torch.max(logits, 1)
                            loss = F.cross_entropy(logits, labels, reduction='mean')
                        
                        

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            scaler.scale(loss).backward()
                            scaler.step(optimizer)
                            scaler.update()
                            optimizer.zero_grad()
                            scheduler.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'valid' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
model_tr = train_model(model,optimizer,scaler, scheduler,NUM_EPOCHS)

torch.save(model_tr.state_dict(), 'ayurmodelstate.pth')

In [ ]:
preds = []
model.eval()
with torch.inference_mode():
    for features, target in test_loader:
        features = features.to(DEVICE)
        target = target.to(DEVICE)
        
        with autocast():
            logits = model(features)
            
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1).detach().cpu()
        
        for pred in y_pred:
            preds.append(pred.item())

In [ ]:
import os
import csv


folder_path = "/kaggle/working/Ayurved-2/test/Test"


file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


csv_file = "image_species.csv"

with open(csv_file, mode="w", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file", "species"])  # Write header row

    for file_name in file_names:
        writer.writerow([file_name, "neem"])  # Write file name and species for each picture

print(f"CSV file '{csv_file}' created with image names and species 'neem'.")


In [ ]:
map_labels = {}
for idx, i in enumerate(class_labels):
    map_labels[idx] = i
    
sub_df = pd.read_csv(SUB_DIR)
sub_df['file'] = test_df['file_path']
sub_df['species'] = pd.Series(preds).map(map_labels)
sub_df.to_csv('submisson.csv', index=False)
print('Submission file saved!')
sub_df

In [ ]:
model = EffNet(num_classes=NUM_CLASSES, embedding_size=EMBEDDING_SIZE)
model.load_state_dict(torch.load(W_PATH))
img_path= '/kaggle/input/test-file-model/dumy.jpg'

def convert_4_channel_to_3_channel(image):
    """
    Convert 4-channel RGBA image to 3-channel RGB image
    """
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    return image

my_transforms = transforms.Compose([
        transforms.Lambda(convert_4_channel_to_3_channel),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image = Image.open(img_path)
image = my_transforms(image)

model.to(DEVICE)
model.eval()

with torch.inference_mode():
    image = image.to(DEVICE)
    image = image.unsqueeze(0)
    with autocast():
        logits = model.forward(image)
            
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1).detach().cpu()
        
    print(y_pred)
    
    predicted_labels = class_labels[y_pred.item()]
    print(predicted_labels)